In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
import json

In [ ]:
# Register local path for the COCO validation dataset
coco_val_path = "/kaggle/input/mscoco/val2017/val2017"
coco_annotations_path = "/kaggle/input/mscoco/annotations_trainval2017/annotations/instances_val2017.json"

register_coco_instances("coco_val", {}, coco_annotations_path, coco_val_path)

In [ ]:
# Function to setup config
def setup_config(output_dir, weights_path=None):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.WEIGHTS = weights_path
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
    cfg.DATASETS.TEST = ("coco_val",)
    cfg.OUTPUT_DIR = output_dir
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    return cfg

In [ ]:
# Function to evaluate the model and save results
def evaluate_and_save(cfg, evaluator_output_file):
    model = DefaultPredictor(cfg).model
    evaluator = COCOEvaluator("coco_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
    val_loader = build_detection_test_loader(cfg, "coco_val")
    print(f"Evaluating model with weights: {cfg.MODEL.WEIGHTS}")
    metrics = inference_on_dataset(model, val_loader, evaluator)

    # Save metrics to a JSON file
    with open(evaluator_output_file, "w") as f:
        json.dump(metrics, f, indent=4)

    print(f"Evaluation results saved to: {evaluator_output_file}")
    return metrics

In [ ]:
# Evaluate pre-trained model
pretrained_cfg = setup_config(
    output_dir="/kaggle/working/output_pretrained/",
    weights_path=model_zoo.get_checkpoint_url("Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml")
)
pretrained_metrics = evaluate_and_save(pretrained_cfg, "pretrained_metrics.json")

# Load and display pre-trained metrics
print("\nPre-trained Model Metrics:")
print(json.dumps(pretrained_metrics, indent=4))

In [ ]:
# Evaluate final trained model
trained_cfg = setup_config(
    output_dir="/kaggle/working/output_trained/",
    weights_path="/kaggle/input/model_final.pth"
)
trained_metrics = evaluate_and_save(trained_cfg, "trained_metrics.json")

# Load and display trained model metrics
print("\nTrained Model Metrics:")
print(json.dumps(trained_metrics, indent=4))